In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer


In [4]:
dftrain = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')
dftrain.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
dftest= dftest.drop(columns=['keyword', 'location'])

In [6]:
dftrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
dftrain = dftrain.drop(columns=['location','id'])


In [8]:
dftrain.tail()

,keyword,text,target
7608,NaN,Two giant cranes holding a bridge collapse int...,1
7609,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,NaN,Police investigating after an e-bike collided ...,1
7612,NaN,The Latest: More Homes Razed by Northern Calif...,1


In [9]:
lem = WordNetLemmatizer()
stemer = PorterStemmer()

def lowlema(txt):
  txt =txt.lower()
  txt = word_tokenize(txt)
  for i in txt:
    if i in stopwords.words("english"):
      txt.remove(i)
  for i in range(len(txt)):
      txt[i] = stemer.stem(txt[i])
  txt = ' '.join(txt)  
  txt = lem.lemmatize(txt)   
  return txt

dftrain['text']=dftrain['text'].apply(lowlema)
dftest['text'] = dftest['text'].apply(lowlema)

In [10]:
dftrain['text'] = dftrain['text'].str.replace(r'[^\w\s]+', '')
dftest['text'] = dftest['text'].str.replace(r'[^\w\s]+', '')

C:\Users\Akash\AppData\Local\Temp\ipykernel_14108\544768780.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  dftrain['text'] = dftrain['text'].str.replace(r'[^\w\s]+', '')
C:\Users\Akash\AppData\Local\Temp\ipykernel_14108\544768780.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dftest['text'] = dftest['text'].str.replace(r'[^\w\s]+', '')


In [11]:
xtrain =dftrain['text']
ytrain = dftrain['target']
xtest = dftest['text']


In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
vec = CountVectorizer()
tftransfor = TfidfTransformer()
tfvector = TfidfVectorizer()

In [13]:
X_train_tf = tfvector.fit(xtrain)
X_train_tf = tfvector.transform(xtrain)
X_test_tf = tfvector.transform(xtest)

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [16]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_tf.shape[1],)))
model.add(Dropout (0.3))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))


In [18]:
model.compile(loss = 'binary_crossentropy', optimizer = Adam(learning_rate=0.01),metrics=['accuracy'])

In [19]:
model.fit(X_train_tf.toarray(), ytrain, batch_size=32, epochs=10,validation_split=0.2)

Epoch 1/10
191/191 [==============================] - 11s 50ms/step - loss: 0.5144 - accuracy: 0.7558 - val_loss: 0.4799 - val_accuracy: 0.7774
Epoch 2/10
191/191 [==============================] - 9s 48ms/step - loss: 0.2358 - accuracy: 0.9074 - val_loss: 0.5293 - val_accuracy: 0.7663
Epoch 3/10
191/191 [==============================] - 9s 48ms/step - loss: 0.1040 - accuracy: 0.9619 - val_loss: 0.7585 - val_accuracy: 0.7262
Epoch 4/10
191/191 [==============================] - 9s 47ms/step - loss: 0.0569 - accuracy: 0.9770 - val_loss: 1.0183 - val_accuracy: 0.7131
Epoch 5/10
191/191 [==============================] - 9s 47ms/step - loss: 0.0513 - accuracy: 0.9816 - val_loss: 1.1792 - val_accuracy: 0.7203
Epoch 6/10
191/191 [==============================] - 9s 47ms/step - loss: 0.0301 - accuracy: 0.9878 - val_loss: 1.0990 - val_accuracy: 0.7144
Epoch 7/10
191/191 [==============================] - 9s 48ms/step - loss: 0.0306 - accuracy: 0.9885 - val_loss: 1.0308 - val_accuracy: 0.714

In [22]:
predicted_probs = model.predict(X_test_tf.toarray())
predicted_df = pd.DataFrame({
    'Text': X_test_tf,  # Assuming 'text_test' is the text data from your test set
    'PredictedProbability': predicted_probs.flatten(),
    'PredictedLabel': (predicted_probs >= 0.5).astype(int).flatten()  # Convert probabilities to binary labels
})

102/102 [==============================] - 1s 5ms/step


In [23]:
predicted_df

,Text,PredictedProbability,PredictedLabel
0,"(0, 17512)\t0.6446980108734724\n (0, 6046)\...",0.003534,0
1,"(0, 12369)\t0.3747848457327939\n (0, 11322)...",0.495660,0
2,"(0, 12469)\t0.3069864577200327\n (0, 12264)...",0.999956,1
3,"(0, 18901)\t0.3963389091520933\n (0, 12252)...",0.999950,1
4,"(0, 18170)\t0.38634499376493414\n (0, 12749...",0.999995,1
...,...,...,...
3258,"(0, 19439)\t0.31237997650442323\n (0, 11326...",0.956960,1
3259,"(0, 19193)\t0.31249446253735064\n (0, 19039...",1.000000,1
3260,"(0, 7870)\t0.4955719986505727\n (0, 6455)\t...",0.999994,1
3261,"(0, 18753)\t0.30507847179043046\n (0, 13125...",0.999982,1


In [26]:
dftest['target'] = predicted_df['PredictedLabel']

In [28]:
dftest= dftest.drop(columns = ['text'])

In [29]:
dftest

,id,target
0,0,0
1,2,0
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [30]:
dftest.to_csv('predicted_results.csv', index=False)